# Examples

## Data

In [ ]:
%matplotlib inline
import numpy as np
from linearmodels.datasets import munnell
data = munnell.load()

regions = {
    'GF':['AL', 'FL', 'LA', 'MS'],
    'MW':['IL', 'IN', 'KY', 'MI', 'MN', 'OH', 'WI'],
    'MA':['DE', 'MD', 'NJ', 'NY', 'PA', 'VA'],
    'MT' :['CO', 'ID', 'MT', 'ND', 'SD', 'WY'],
    'NE' :['CT', 'ME', 'MA', 'NH', 'RI', 'VT'],
    'SO' :['GA', 'NC', 'SC', 'TN', 'WV', 'AR'],
    'SW' : ['AZ', 'NV', 'NM', 'TX', 'UT'],
    'CN': ['AK', 'IA','KS', 'MO','NE','OK'],
    'WC': ['CA','OR','WA']
}

def map_region(state):
    for key in regions:
        if state in regions[key]:
            return key


data['REGION'] = data.ST_ABB.map(map_region)
data['TOTAL_EMP'] = data.groupby(['REGION','YR'])['EMP'].transform('sum')
data['EMP_SHARE'] = data.EMP / data.TOTAL_EMP
data['WEIGHED_UNEMP'] = data.EMP_SHARE * data.UNEMP

In [ ]:
grouped = data.groupby(['REGION','YR'])
agg_data = grouped[['GSP','PC','HWY','WATER','UTIL','EMP','WEIGHED_UNEMP']].sum()
for col in ['GSP','PC','HWY','WATER','UTIL','EMP']:
    agg_data['ln'+col] = np.log(agg_data[col])
agg_data['UNEMP'] = agg_data.WEIGHED_UNEMP
agg_data['Intercept'] = 1.0

## Basic Usage

In [ ]:
from collections import OrderedDict
mod_data = OrderedDict()
for region in ['GF','SW','WC','MT','NE','MA','SO','MW','CN']:
    region_data = agg_data.loc[region]
    dependent = region_data.lnGSP
    exog = region_data[['Intercept', 'lnPC', 'lnHWY', 'lnWATER', 'lnUTIL', 'lnEMP', 'UNEMP']]
    mod_data[region] = {'dependent': dependent, 'exog': exog}

In [ ]:
import pandas as pd
from linearmodels.system import SUR
mod = SUR(mod_data)
res = mod.fit(cov_type='unadjusted', method='gls', debiased=True, iter_limit=10)
cov = res.sigma
std = np.sqrt(np.diag(res.sigma)[:,None])
regions =  [k for k in mod_data.keys()]
corr = pd.DataFrame(cov / (std @ std.T), columns=regions, index=regions)

import seaborn as sns
import matplotlib.pyplot as plt
sns.heatmap(corr, vmax=.8, square=True)
plt.show()

corr.style.format('{:0.3f}')

In [ ]:
from IPython.display import Image, display_png
display_png(Image('correct-greene-table-10-2.png'))

In [ ]:
from linearmodels.iv import IV2SLS
ols_res = IV2SLS(mod_data['WC']['dependent'], mod_data['WC']['exog'], None,None).fit(cov_type='unadjusted',debiased=True)
print(ols_res)
res.equations['WC'].tstats - ols_res.tstats

In [ ]:
from linearmodels.panel import PooledOLS
pooled_mod = PooledOLS(agg_data.lnGSP, agg_data[['Intercept','lnPC','lnHWY','lnWATER','lnUTIL','lnEMP','UNEMP']])
print(pooled_mod.fit())

In [ ]:
res.equations['GF'].params

In [ ]:
display_png(Image('correct-greene-table-10-1.png'))

## Estimation Options

### Restricted Residual Covariance

### Iterative GLS

### Alternative Covariance Estimators

## Pre-specified Residual Covariance Estimators

## Multivariate OLS

## Cross-Equation Restrictions